In [19]:
import feedparser
import requests
import re
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json

from tqdm import tqdm

# Etape 1 : Extraction des flux RSS

In [20]:
def extract_rss_entries():
    urls = [
        "https://www.cert.ssi.gouv.fr/avis/feed",
        "https://www.cert.ssi.gouv.fr/alerte/feed"
    ]
    entries = []
    for url in urls:
        feed = feedparser.parse(url)
        for entry in feed.entries:
            entries.append({
                'id': entry.link.split("/")[-2],
                'title': entry.title,
                'link': entry.link,
                'type': "Alerte" if "alerte" in url else "Avis",
                'date': entry.published
            })
    return entries

In [21]:
rss_entries = extract_rss_entries()
print(f"Nombre d'entrées RSS : {len(rss_entries)}\n")
# print(rss_entries)

Nombre d'entrées RSS : 80



# Etape 2 : Extraction des CVE

In [64]:
def extract_cves_from_json(entry_link):
    try:
        json_url = entry_link + "/json/"
        response = requests.get(json_url)
        data = response.json() # fichier JSON complet de l'entrée RSS

        # Extraction des CVE reference dans la clé cves du dict data
        cve_refs = data.get("cves", [])

        # Attention, il s’agit d’une liste des dictionnaires avec name et url comme clés
        print("CVE références :", cve_refs)
        print("Type de références", type(cve_refs))
        print("Type des éléments de la liste", type(cve_refs[0]))

        # Extraction des CVE avec une regex
        cve_pattern = r"CVE-\d{4}-\d{4,7}"
        cve_list = list(set(re.findall(cve_pattern, str(data))))
        print("CVE trouvés :", cve_list)

        # Retourne la liste de CVE référence pour l'entrée RSS entrée en argument + le JSON complet de l'entrée RSS
        return cve_refs, data
    except:
        return [], {}


In [65]:
first_entry = rss_entries[0]
cve_refs, full_json = extract_cves_from_json(first_entry["link"])
"""print(cve_refs)
print()
print(full_json)"""

"""for entry in rss_entries:
    cve_refs, full_json = extract_cves_from_json(entry["link"])
    print(cve_refs)
    print(full_json)"""

CVE références : [{'name': 'CVE-2025-4128', 'url': 'https://www.cve.org/CVERecord?id=CVE-2025-4128'}, {'name': 'CVE-2025-4573', 'url': 'https://www.cve.org/CVERecord?id=CVE-2025-4573'}]
Type de références <class 'list'>
Type des éléments de la liste <class 'dict'>
CVE trouvés : ['CVE-2025-4128', 'CVE-2025-4573']


'for entry in rss_entries:\n    cve_refs, full_json = extract_cves_from_json(entry["link"])\n    print(cve_refs)\n    print(full_json)'

# Etape 3 : Enrichissement des CVE

In [66]:
def enrich_cve(cve_id):
    time.sleep(2)
    mitre_data = {}
    epss_score = None

    try:
        mitre_url = f"https://cveawg.mitre.org/api/cve/{cve_id}"
        r = requests.get(mitre_url)
        mitre_data = r.json()
    except:
        pass

    try:
        epss_url = f"https://api.first.org/data/v1/epss?cve={cve_id}"
        r = requests.get(epss_url)
        epss_json = r.json()
        epss_score = epss_json.get("data", [{}])[0].get("epss")
    except:
        epss_score = None

    return mitre_data, epss_score